# Mobility data analysis of researchers trajectories

*Author: Liubov*; *Collaborators: Marc, Bastian, Vero*

Notebook inspired from Open humans project https://www.openhumans.org

Read mobility files from Open humans
https://www.openhumans.org/api/public-data/?source=direct-sharing-138

Other notebooks 
https://exploratory.openhumans.org/notebooks/?source=Moves%20connection

Openhumans open API 
https://open-humans-api.readthedocs.io/en/latest/cli.html#example-use-cases


# Content 
1. Data preprocessing
2. Data analysis
3. Data visualisation
# 1. Data preprocessing

Json data analysis is given here:
https://www.dataquest.io/blog/python-json-tutorial/ 
# Transforming json file to dataframe

We have json file with google locations and we need to transform it to dataframe.

In [1]:
# main function analyzing moves_data    
# starting date of being researcher - get is from google form CSV file


#DATARANGE_START = "2016-06-01"
#DATARANGE_END = "2018-05-08"

import os
import json
import requests
from datetime import datetime
from collections import defaultdict
import pandas as pd
import numpy as np
import seaborn as sns


# sets the axis label sizes for seaborn
rc={'font.size': 14, 'axes.labelsize': 14, 'legend.fontsize': 14.0, 
    'axes.titlesize': 14, 'xtick.labelsize': 14, 'ytick.labelsize': 14}
sns.set(rc=rc)

#with open('C:/Users/lyubo/Documents/DATA_networks/mobilitydata/openhumans/moves-storyline-data.json') as f:
#    moves_data = json.load(f)

#with open('C:/Users/lyubo/Documents/DATA_networks/mobilitydata/openhumans/moves-storyline-data98972.json') as f:
#    moves_data3 = json.load(f)   
    

id_research1 = "05364098" # Bastian ID 05364098
#id_research1 = "32891125"#ID of researcher  # Marc ID 32891125 
print('data loaded for researcher ', str(id_research1))

with open('C:/Users/lyubo/Documents/DATA_networks/mobilitydata/openhumans/researcher_mobility/'+id_research1+'/direct-sharing-182/Location History.json') as f:
    moves_data_researcher1 = json.load(f)  
#with open('C:/Users/lyubo/Documents/DATA_networks/mobilitydata/openhumans/researcher_mobility/'+id_research1+'/direct-sharing-182/Location History.json') as f:
#    moves_data_researcher1 = json.load(f)   



df1 = pd.DataFrame(moves_data_researcher1)
df1.head(10)
#print(type(df1_datapoint ))
#print(df1.columns)


# function to get data from json data
#df1 = dataframe_from_json(moves_data) # works for other geo-files 
    
df1.locations.iloc[1]

data loaded for researcher  05364098


,locations
0,"{'timestampMs': '1280557864967', 'latitudeE7':..."
1,"{'timestampMs': '1280557901000', 'latitudeE7':..."
2,"{'timestampMs': '1280557967983', 'latitudeE7':..."
3,"{'timestampMs': '1280557969000', 'latitudeE7':..."
4,"{'timestampMs': '1280558011227', 'latitudeE7':..."
5,"{'timestampMs': '1280558133517', 'latitudeE7':..."
6,"{'timestampMs': '1280558311735', 'latitudeE7':..."
7,"{'timestampMs': '1280558591706', 'latitudeE7':..."
8,"{'timestampMs': '1280558732806', 'latitudeE7':..."
9,"{'timestampMs': '1280558893110', 'latitudeE7':..."


Moves data for a researcher.

In [ ]:
"'locations': [{'timestampMs': '1546429982000',
   'latitudeE7': 119987508,
   'longitudeE7': 1202035421,
   'accuracy': 5,
   'velocity': 1,
   'heading': 240,
   'altitude': 11,
   'verticalAccuracy': 4},


# Creating new data frame and readable csv file
We need to split column values into different values: timestamps, latitude, longitude etc.
This file we can then visualised in deck.gl or kepler.gl .

We can also later upload it to Openhumans notebook.


In [2]:

# create dataframe with lat, lon
time_array = np.zeros(int(df1.shape[0]))
lat_array = np.zeros(int(df1.shape[0]))
lon_array = np.zeros(int(df1.shape[0]))

 
for ind in range(0,int(df1.shape[0])): # make a loop in all rows of dataframe df1 
    #df1_datapoint = df1.iloc[ind].values
    dict_df1_data = df1.iloc[ind].values[0]
    
    time_array[ind] = dict_df1_data['timestampMs']
    lat_array[ind] = dict_df1_data['latitudeE7']*0.0000001
    lon_array[ind] = dict_df1_data['longitudeE7']*0.0000001

print(lon_array[ind])


# adding new columns
df1['time'] = time_array
df1['lat'] = lat_array
df1['lon'] = lon_array


print('done with creating new dataframe')
    
# get ith datapoint in dataframe: df1_datapoint = df1.iloc[i].values
# get value in dictionary for timestamp, lat, lon: dict_df1_data = df1_datapoint[0]
# dict_df1_data['timestampMs'] ... 

2.3655999999999997
done with creating new dataframe


 Let's see how the data looks like now.

In [5]:

df1.head(20)

,locations,time,lat,lon
0,"{'timestampMs': '1280557864967', 'latitudeE7':...",1.280558e+12,52.325120,8.097732
1,"{'timestampMs': '1280557901000', 'latitudeE7':...",1.280558e+12,52.536894,8.113194
2,"{'timestampMs': '1280557967983', 'latitudeE7':...",1.280558e+12,52.557340,8.113479
3,"{'timestampMs': '1280557969000', 'latitudeE7':...",1.280558e+12,52.561325,8.112839
4,"{'timestampMs': '1280558011227', 'latitudeE7':...",1.280558e+12,52.572300,8.110485
5,"{'timestampMs': '1280558133517', 'latitudeE7':...",1.280558e+12,52.612850,8.125504
6,"{'timestampMs': '1280558311735', 'latitudeE7':...",1.280558e+12,52.636900,8.150499
7,"{'timestampMs': '1280558591706', 'latitudeE7':...",1.280559e+12,52.585690,8.117764
8,"{'timestampMs': '1280558732806', 'latitudeE7':...",1.280559e+12,52.591050,8.217593
9,"{'timestampMs': '1280558893110', 'latitudeE7':...",1.280559e+12,52.649680,8.158476


In [6]:
# save file to csv file

df1.to_csv('df1_ID_'+str(id_research1)+'_researcher.csv')#id_researcher1

# 2. Data visualisation 

Visualise trajectory on the map.

We make the visualisation on a map 

We put this to OH account https://open-humans-api.readthedocs.io/en/latest/cli.html#upload-files-to-the-accounts-of-your-project-members 

In [22]:
import numpy as np
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import folium

# function to visualise trajectories on a map 

def visualise_df_traj(df):#, lat_values, lon_values):
    ''' function to visusalise trajectories on a map
    input:
    lat_values = df.latitude.values
    lon_values = df.longitude.values'''
    
    latitude = df.lat.values
    longitude = df.lon.values

    #then we zip two arrays of lat, lon of datapoints
    latlon1 = list(zip(latitude, longitude))


    #We put map to show first location of Paris 48.8566° North, 2.3522° est
    mapit = folium.Map( location=[48.75 , 2.35], zoom_start=6 )
    for coord in latlon1:
        folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=8 ).add_to( mapit )
    mapit


    #draw connecting line between locations written in latlon file

#    trips = df.tripid # different trips
#    array_trip = np.unique(trips) 
    SAF=folium.PolyLine(locations=latlon1,weight=5,color = 'r')#color_array[ind])
    mapit.add_child(SAF)
    #do not use plt.show with folium!
    


In [ ]:
#run function
lat_values = df1.lat.values
lon_values = df1.lon.values

visualise_df_traj(df1)#, lat_values, lon_values)




In [ ]:
# use folium map without function 
# attention Folium map may be quite heave when you run it

latitude = df1.lat.values
longitude = df1.lon.values

#then we zip two arrays of lat, lon of datapoints
latlon1 = list(zip(latitude, longitude))


#we put map to show first location of Paris 48.8566° North, 2.3522° est
mapit = folium.Map( location=[48.75 , 2.35], zoom_start=6 )
for coord in latlon1:
    folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=8 ).add_to( mapit )
mapit


#draw connecting line between locations written in latlon file

SAF=folium.PolyLine(locations=latlon1,weight=5,color = 'r')#color_array[ind])
mapit.add_child(SAF)
#plt.show()







# 3. Data analysis

Here we do trajectories analysis of researchers and user profiling:

0. we analyze distributions of trip durations, trip lengths 
1. we analyze frequencies of cities and places visited
2. we calculate the number of amenities of places around the trajectories in places, where researchers stayed the longest
3. we analyze possible common properties of trajectories of researchers




In [1]:
#functions inserted from OH notebook 

import json
from pprint import pprint


'''
Functions:
 to estimate the distribution of stop duration
 to estimate the distribution of jump length
'''



def len_jumps_distribution(daily_segments):
    """
    takes a daily segment of Moves 
    returns the distribution of jumps lengths (calculated from lat/long of stops)
    can be misleading for days w/ lots of travel etc. 
    """
    places_of_traj = [] #returns array of all locations during the day
    for i in daily_segments: 
        if i['type'] == 'place':
            place_location = i['place']['location']
            places_of_traj.append([place_location])
    return places_of_traj


def longest_daily_location(daily_segments):
    """
    Function from Bastian (Open Humans)
       
    takes a daily segment log of Moves and returns the 
    lat/long for the location where most time was spent. 
    Can be misleading for days w/ lots of travel etc. 
    But the most quick/dirty solution for now.
    """
    places_of_day = []
    for i in daily_segments:
        if i['type'] == 'place':
            place_location = i['place']['location']
            start_time = datetime.strptime(i['startTime'],'%Y%m%dT%H%M%S%z')
            end_time = datetime.strptime(i['endTime'],'%Y%m%dT%H%M%S%z')
            duration = end_time - start_time
            places_of_day.append([place_location,duration])
    places_of_day.sort(key=lambda tup: tup[-1],reverse=True)
    return places_of_day[0][0]


# Data analysis preprocessing

We create additional column in dataframe.

In [3]:
from math import sin, cos, sqrt, atan2, radians



# analysis of trip durations and distances

def dist_lat_lon(lat1, lat2, lon1, lon2):
    # approximate radius of earth in km
    R = 6373.0

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return distance

size = df1.shape
print("size of dataframe ", size[0])

distances_array = np.zeros(int(size[0]))



for ind in range(0,int(size[0])-1):

    lat1 = df1.lat.iloc[ind]
    lon1 = df1.lon.iloc[ind]
    lat2 = df1.lat.iloc[ind+1]
    lon2 = df1.lon.iloc[ind+1]
    distances_array[ind] = dist_lat_lon(lat1, lat2, lon1, lon2)

    
print('done with preprocessing the data of distances between locations')    
# adding new columns
df1['distance'] = distances_array

array_distance = df1.distance.values 
type(array_distance)

# analysis of main locations, where person spent significant amount of time 

# analysis of places and amenities around the main locations (see other notebook)


size of dataframe  136571


numpy.ndarray


# Alternative interactive visualisation

We use kepler.gl for plotting the data. For this one would need kepler.gl installed or you can run generated file on kepler.gl website in the browser.

Alternative visualisations can be done using
1. https://mode.com/blog/python-interactive-plot-libraries
2. https://github.com/keplergl/kepler.gl

In [ ]:
import keplergl

# Load a map with data and config and height
from keplergl import KeplerGl
map_1 = KeplerGl(height=400, data={"data_1": my_df}, config=config)

# DataFrame is loaded

# CSV file loading
with open('df1_ID_'+str(id_research1)+'_researcher.csv', 'r') as f:
    csvData = f.read()
map_1.add_data(data=csvData, name='data_2')

map_1.add_data(data=geojson, name='geojson')